In [632]:
!pip install pandas; numpy; psycopg2

In [633]:
# importing needed libraries
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime, timedelta
import csv

In [634]:
# reading the dataset from the source(CSV)
zulo_bank_df = pd.read_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\zulo_bank.csv')

In [635]:
# Viewing the first five rows of the dataset
zulo_bank_df.head()

,FullName,Email,Phone,TransactionType,Amount,TransactionDate,AccountType,Balance,OpeningDate,LoanAmount,LoanType,StartDate,EndDate,InterestRate
0,Carol Miller,yfisher@example.org,6088279027,withdrawal,102.15,2023-04-26,Savings,5652.16,2019-08-12,NaN,NaN,NaN,NaN,NaN
1,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,32428.90,Mortgage,2021-06-24,2050-01-08 04:59:17.907588,2.12
2,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,31406.77,Personal,2021-02-27,2038-10-12 04:59:17.907821,4.63
3,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,27834.00,Personal,2019-12-05,2037-08-15 04:59:17.909497,2.15
4,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,27873.08,Auto,2022-01-19,2037-06-03 04:59:17.913974,7.03


In [636]:
zulo_bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FullName         1554 non-null   object 
 1   Email            1554 non-null   object 
 2   Phone            1554 non-null   object 
 3   TransactionType  1554 non-null   object 
 4   Amount           1554 non-null   float64
 5   TransactionDate  1554 non-null   object 
 6   AccountType      1554 non-null   object 
 7   Balance          1554 non-null   float64
 8   OpeningDate      1554 non-null   object 
 9   LoanAmount       1278 non-null   float64
 10  LoanType         1278 non-null   object 
 11  StartDate        1278 non-null   object 
 12  EndDate          1278 non-null   object 
 13  InterestRate     1278 non-null   float64
dtypes: float64(4), object(10)
memory usage: 170.1+ KB


In [637]:
# converting the dat columns from  strings to datetime
zulo_bank_df ['OpeningDate'] = pd.to_datetime(zulo_bank_df['OpeningDate'])
zulo_bank_df ['StartDate'] = pd.to_datetime(zulo_bank_df['StartDate'])
zulo_bank_df ['EndDate'] = pd.to_datetime(zulo_bank_df['EndDate'])
zulo_bank_df ['TransactionDate'] = pd.to_datetime(zulo_bank_df['TransactionDate'])

In [638]:
zulo_bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   FullName         1554 non-null   object        
 1   Email            1554 non-null   object        
 2   Phone            1554 non-null   object        
 3   TransactionType  1554 non-null   object        
 4   Amount           1554 non-null   float64       
 5   TransactionDate  1554 non-null   datetime64[ns]
 6   AccountType      1554 non-null   object        
 7   Balance          1554 non-null   float64       
 8   OpeningDate      1554 non-null   datetime64[ns]
 9   LoanAmount       1278 non-null   float64       
 10  LoanType         1278 non-null   object        
 11  StartDate        1278 non-null   datetime64[ns]
 12  EndDate          1278 non-null   datetime64[ns]
 13  InterestRate     1278 non-null   float64       
dtypes: datetime64[ns](4), float64(4), object

In [639]:
zulo_bank_df.isnull().sum()

FullName             0
Email                0
Phone                0
TransactionType      0
Amount               0
TransactionDate      0
AccountType          0
Balance              0
OpeningDate          0
LoanAmount         276
LoanType           276
StartDate          276
EndDate            276
InterestRate       276
dtype: int64

In [640]:
# Filling the null values with approprite values
zulo_bank_df['LoanAmount'] = zulo_bank_df['LoanAmount'].fillna(0)
zulo_bank_df['LoanType'] = zulo_bank_df['LoanType'].fillna('Not Provided')
zulo_bank_df['InterestRate'] = zulo_bank_df['InterestRate'].fillna(0)

In [641]:
# converting from denormallized to 1NF
zulo_bank_df [['First_Name', 'Last_Name']] = zulo_bank_df['FullName'].str.split(expand=True)

In [642]:
#converting from 1NF to 2NF
transaction_df = zulo_bank_df[['TransactionType', 'Amount','TransactionDate']].copy().drop_duplicates().reset_index(drop =True)
transaction_df['TransactionID'] = range(1, len(transaction_df) + 1)
transaction_df = transaction_df[['TransactionID', 'TransactionType', 'Amount','TransactionDate']]

In [643]:
customer_df = zulo_bank_df [['First_Name', 'Last_Name', 'Email', 'Phone']].copy().drop_duplicates().reset_index(drop = True)
customer_df['CustomerID'] = range(1, len(customer_df) + 1)
customer_df = customer_df[['CustomerID','First_Name', 'Last_Name', 'Email', 'Phone']]

In [644]:
account_df = zulo_bank_df[['AccountType', 'Balance', 'OpeningDate']].copy().drop_duplicates().reset_index(drop =True)
account_df['AccountID'] = range(1, len(account_df)+ 1)
account_df = account_df[['AccountID', 'AccountType', 'Balance', 'OpeningDate']]

In [645]:
loan_df = zulo_bank_df[['LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate']]. copy().drop_duplicates().reset_index(drop =True)
loan_df['LoanID'] = range(1, len(loan_df) + 1)
loan_df = loan_df[['LoanID','LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate']]

In [646]:
# using the merge operation to create a comprehebsive zulo_bank table with the IDs
zulo_bank_df = zulo_bank_df.merge(customer_df, on  =['First_Name', 'Last_Name', 'Email', 'Phone'], how = 'left')\
                            .merge(account_df, on= ['AccountType', 'Balance', 'OpeningDate'], how = 'left')\
                            .merge(loan_df, on = ['LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate'], how = 'left')\
                            .merge(transaction_df, on = ['TransactionType', 'Amount','TransactionDate'])\
                            [['CustomerID', 'AccountID', 'LoanID','TransactionID']]

In [669]:
zulo_bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CustomerID     1554 non-null   int64
 1   AccountID      1554 non-null   int64
 2   LoanID         1554 non-null   int64
 3   TransactionID  1554 non-null   int64
dtypes: int64(4)
memory usage: 48.7 KB


In [648]:
# converting from 2NF to 3NF by creating a date table for all date columns to avoid transititive dependency
start_date = datetime(2020, 1, 1)
current_date= datetime(2050, 12, 31)

#number of days between start date and current date
num_days =(current_date - start_date).days

#getting a list of dates from start date to the current date
date_list = [start_date + timedelta(days =x) for x in range(num_days+1)] # why time delta and not datetime

#making sure date_id matches the length of the date_df
date = {'DateID':[x for x in  range(1, len(date_list) + 1)], 'date': date_list}

date_dim = pd.DataFrame(date)
date_dim['Day'] = date_dim['date'].dt.day
date_dim['Month'] = date_dim['date'].dt.month
date_dim['Year'] = date_dim['date'].dt.year


In [649]:
date_dim.head()

,DateID,date,Day,Month,Year
0,1,2020-01-01,1,1,2020
1,2,2020-01-02,2,1,2020
2,3,2020-01-03,3,1,2020
3,4,2020-01-04,4,1,2020
4,5,2020-01-05,5,1,2020


In [650]:
account_df = account_df.merge(date_dim, left_on = 'OpeningDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID': 'OpeningDateID'})\
                        .reset_index (drop = True)\
                        [['AccountID', 'AccountType', 'Balance', 'OpeningDateID']]

In [651]:
transaction_df = transaction_df.merge(date_dim, left_on ='TransactionDate', right_on = 'date', how = 'inner')\
                                .rename(columns = {'DateID' : 'TransactionDateID'})\
                                .reset_index (drop =True)\
                                [['TransactionID', 'TransactionType', 'Amount','TransactionDateID']]


In [652]:
loan_df = loan_df.merge(date_dim, left_on = 'StartDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID' : 'StartDateID'})\
                        .merge(date_dim, left_on = 'EndDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID' : 'EndDateID'})\
                        .reset_index (drop =True)\
                        [['LoanID', 'LoanAmount', 'LoanType', 'StartDateID', 'EndDateID','InterestRate']]


In [653]:
# zulo_bank_df = zulo_bank_df.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\zulo_fact.csv', index = False)

In [654]:
# transaction_df = transaction_df.to_csv( r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction.csv',index = False)
# account_df = account_df.to_csv( r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\account.csv', index = False)
# customer_df = customer_df.to_csv( r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\customer.csv', index = False)
# loan_df = loan_df.to_csv( r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan.csv', index = False)
# date_dim = date_dim.to_csv( r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\date_dim.csv', index = False)

In [655]:
#creating models for my datwarehouse
# transaction fact table
#loan fact table
transaction_dim = transaction_df[['TransactionID','TransactionType']].copy().drop_duplicates().reset_index(drop =True)

account_dim = account_df [['AccountID', 'AccountType', 'Balance']].copy().drop_duplicates().reset_index(drop =True)

transaction_fact = zulo_bank_df.merge(transaction_df, on = 'TransactionID', how = 'inner')\
                                .merge(account_df, on = 'AccountID', how = 'inner')\
                                [['TransactionID', 'AccountID', 'TransactionDateID', 'OpeningDateID', 'Amount']]



In [656]:
loan_dim = loan_df[['LoanID', 'LoanType']].copy().drop_duplicates().reset_index(drop = True)
customer_dim = customer_df[['First_Name', 'Last_Name', 'Email', 'Phone']].copy().drop_duplicates().reset_index(drop = True)
loan_fact = zulo_bank_df.merge(loan_df, on = 'LoanID', how = 'inner')\
                        .merge(customer_df, on = 'CustomerID', how = 'inner')\
                        [['LoanID', 'CustomerID', 'StartDateID', 'EndDateID', 'LoanAmount', 'InterestRate']]

In [657]:
# transaction_dim = transaction_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction_dim.csv', index = False)
# account_dim = account_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\account_dim.csv', index = False)
# loan_dim = loan_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan_dim.csv', index = False)
# customer_dim = customer_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\customer_dim.csv', index = False)
# loan_fact = loan_fact.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan_fact.csv', index = False)
# transaction_fact = transaction_fact.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction_fact.csv', index = False)

In [658]:
# loading my data into my databse(Postgres)
db_params = {
    'user': 'postgres',
    'password':'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'zulobank'
}

default_db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/postgres"
try:
    conn = psycopg2.connect(default_db_url)
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_params['database']}'")
    exists = cursor.fetchone()
    if not exists:
        cursor.execute(f'CREATE DATABASE {db_params['database']}')
        print (f'Database {db_params['database']} creted successfully')
    else:
        print(f'Database {db_params['database']} already exists')
    cursor.close()
    conn.close()

except Exception as e:
    print(f'An erroe has occurred: {e}')


Database zulobank already exists


In [659]:
#Connecting to the creted database(zulobank) using the engine

def get_db_connection():
    connection = psycopg2.connect(
    host = 'localhost',
    database = 'zulobank',
    user = 'postgres',
    password ='postgres'
    )
    return connection
conn = get_db_connection()


In [660]:
# creating the required schema and tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    table_query = '''
                CREATE SCHEMA IF NOT EXISTS zulobankdb; 
                DROP TABLE IF EXISTS zulobankdb.customers CASCADE;
                DROP TABLE IF EXISTS zulobankdb.accounts CASCADE;
                DROP TABLE IF EXISTS zulobankdb.transactions CASCADE;
                DROP TABLE IF EXISTS zulobankdb.loans CASCADE;
                DROP TABLE IF EXISTS zulobankdb.date_dim CASCADE;
                DROP TABLE IF EXISTS zulobankdb.zulo_fact CASCADE;


                CREATE TABLE IF NOT EXISTS zulobankdb.date_dim(
                DateID SERIAL PRIMARY KEY,
                date VARCHAR (1000),
                Day INT,
                Month INT,
                Year INT
                
                );

                CREATE TABLE IF NOT EXISTS zulobankdb.transactions(
                TransactionID SERIAL PRIMARY KEY,
                TransactionType VARCHAR (1000),
                Amount FLOAT,
                TransactionDateID INT,
                FOREIGN KEY(TransactionDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );


                CREATE TABLE IF NOT EXISTS zulobankdb.accounts(
                AccountID SERIAL PRIMARY KEY,
                AccountType VARCHAR (1000),
                Balance FLOAT,
                OpeningDateID INT,
                FOREIGN KEY(OpeningDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );

                CREATE TABLE IF NOT EXISTS zulobankdb.loans(
                LoanID SERIAL PRIMARY KEY,
                LoanAmount FLOAT,
                LoanType VARCHAR(1000),
                StartDateID INT,
                EndDateID INT,
                InterestRate FLOAT,
                FOREIGN KEY(StartDateID) REFERENCES zulobankdb.date_dim(DateID),
                FOREIGN KEY(EndDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );


                CREATE TABLE IF NOT EXISTS zulobankdb.customers(
                CustomerID SERIAL PRIMARY KEY,
                First_Name VARCHAR(1000),
                Last_Name VARCHAR(1000),
                Email VARCHAR(1000),
                Phone VARCHAR(1000)
                );
            
                CREATE TABLE IF NOT EXISTS zulobankdb.zulo_fact(
                CustomerID INT,
                AccountID INT,
                LoanID INT,
                TransactionID INT,
                FOREIGN KEY(CustomerID) REFERENCES zulobankdb.customers(CustomerID),
                FOREIGN KEY(AccountID) REFERENCES zulobankdb.accounts(AccountID),
                FOREIGN KEY(LoanID) REFERENCES zulobankdb.loans(LoanID),
                FOREIGN KEY(TransactionID) REFERENCES zulobankdb.transactions(TransactionID)
                );'''
    cursor.execute(table_query)
    conn.commit()
    cursor.close()
    conn.close()




In [661]:
create_tables()

In [662]:
#loading the datasets into the created db tables
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.date_dim (DateID, date, Day, Month, Year) 
                           VALUES (%s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\date_dim.csv'
load_data_from_csv(csv_path)
print('date_dim data loaded successfully')



date_dim data loaded successfully


In [663]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.transactions (TransactionID, TransactionType, Amount, TransactionDateID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction.csv'
load_data_from_csv(csv_path)
print('transactions data loaded successfully')
        

transactions data loaded successfully


In [664]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.accounts (AccountID, AccountType, Balance, OpeningDateID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\account.csv'
load_data_from_csv(csv_path)
print('accounts data loaded successfully')

accounts data loaded successfully


In [665]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.loans (loanID, LoanAmount, LoanType, StartDateID, EndDateID, InterestRate) 
                           VALUES (%s, %s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan.csv'
load_data_from_csv(csv_path)
print('loans data loaded successfully')

loans data loaded successfully


In [666]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.customers (CustomerID, First_Name, Last_Name, Email, Phone) 
                           VALUES (%s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\customer.csv'
load_data_from_csv(csv_path)
print('customers data loaded successfully')

customers data loaded successfully


In [667]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            try:
                cursor.execute(''' INSERT INTO  zulobankdb.zulo_fact (CustomerID, AccountID, LoanID, TransactionID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
        
                
            except psycopg2.IntegrityError as e: # this is for the freign key violation
                print ('error inserting row;', row)
                conn.rollback() # This rolls back the current transaction
            else:
                conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\zulo_fact.csv'
load_data_from_csv(csv_path)
print('zulo_fact data loaded successfully')

error inserting row; ['1', '1', '1', '1']
error inserting row; ['2', '2', '2', '2']
error inserting row; ['2', '2', '3', '2']
error inserting row; ['2', '2', '4', '2']
error inserting row; ['2', '2', '5', '2']
error inserting row; ['3', '3', '6', '3']
error inserting row; ['3', '3', '7', '3']
error inserting row; ['3', '3', '8', '3']
error inserting row; ['4', '4', '9', '4']
error inserting row; ['4', '4', '10', '4']
error inserting row; ['5', '5', '11', '5']
error inserting row; ['6', '6', '12', '6']
error inserting row; ['7', '7', '13', '7']
error inserting row; ['8', '8', '1', '8']
error inserting row; ['9', '9', '1', '9']
error inserting row; ['10', '10', '1', '10']
error inserting row; ['11', '11', '14', '11']
error inserting row; ['11', '11', '15', '11']
error inserting row; ['11', '11', '16', '11']
error inserting row; ['12', '12', '17', '12']
error inserting row; ['13', '13', '18', '13']
error inserting row; ['13', '13', '19', '13']
error inserting row; ['13', '13', '20', '13']

In [668]:
zulo_fact.info()

NameError: name 'zulo_fact' is not defined